# Installs

In [ ]:
%pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 449.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 60.2 MB/s eta 0:00:00



This may take a while

In [ ]:
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
%cd ctcdecode
!pip install . -q
%cd ..

!pip install torchsummaryX -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 18.7 MB/s eta 0:00:00
Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063
Receiving objects: 100% (1102/1102), 782.27 KiB | 5.88 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.c

In [ ]:
'''
If torchsummaryX doesn't work, please run this cell. Alternatively, please refer to Piazza post @209 for more assistance:
'''

!pip install torchsummaryx==1.1.0

  Attempting uninstall: torchsummaryx
    Found existing installation: torchsummaryX 1.3.0
    Uninstalling torchsummaryX-1.3.0:
      Successfully uninstalled torchsummaryX-1.3.0


# Imports

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Kaggle Setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8 -q
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"vedant100","key":"8a4ebd58f68aa5ec43f7cff81c7e80be"}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!kaggle competitions download -c hw3p2asr-s24

100% 3.74G/3.74G [00:30<00:00, 187MB/s]
100% 3.74G/3.74G [00:30<00:00, 133MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data
'''
!unzip -q hw3p2asr-s24.zip
!ls

11-785-s24-hw3p2  ctcdecode  hw3p2asr-s24.zip  sample_data


# Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# Dataset and Dataloader

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]

In [ ]:
# You might want to play around with the mapping as a sanity check here


### Train Data

In [ ]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, PHONEMES):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, 'mfcc')
        self.transcript_dir = os.path.join(root, 'transcript')

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        self.mfccs, self.transcripts = [], []

        self.phonemeindex = {}

        for i, phoneme in enumerate(self.PHONEMES):
            self.phonemeindex[phoneme] = i

        for i in range(len(self.mfcc_files)):
          # Load and normalize MFCCs
          mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
          mean = np.mean(mfcc)
          std = np.std(mfcc)
          mfcc = (mfcc - mean) / std  # Normalize MFCC
          self.mfccs.append(mfcc)

          # Load transcripts and convert phonemes to indices
          transcript = np.load(os.path.join(self.transcript_dir, self.transcript_files[i]))
          transcript = transcript[1:-1]  # Assuming removal of start/end tokens if present
          transcript_indices = [self.phonemeindex[phoneme] for phoneme in transcript]  # Convert phonemes to indices
          self.transcripts.append(transcript_indices)

        self.transcripts = self.transcripts

        self.length = len(self.mfccs)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''


    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = self.mfccs[ind]
        transcript = self.transcripts[ind]
        # print(f"Debug: Transcript at index {ind} is of type {type(transcript)} with value/content {transcript}")
        return torch.FloatTensor(mfcc), torch.tensor(transcript)


    def collate_fn(self,batch):
        # Extract mfccs and transcripts from the batch
        batch_mfcc = []
        for i in batch:
            batch_mfcc.append(i[0])

        batch_transcript = []
        for i in batch:
          batch_transcript.append(i[1])

        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True)
        lengths_mfcc = []
        for i in batch_mfcc:
            lengths_mfcc.append(len(i))

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True)
        lengths_transcript = []
        for i in batch_transcript:
          lengths_transcript.append(len(i))



        # Return the processed batch components
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



In [ ]:
# # Instantiate your dataset class
# dataset = AudioDataset(root='/content/11-785-s24-hw3p2/train-clean-100', PHONEMES=PHONEMES)

# # Print the total number of utterances in the dataset
# print(f"Total number of utterances in the dataset: {len(dataset)}")

# # Get and print the shape of the first utterance and its corresponding transcript
# first_utterance, first_transcript = dataset.__getitem__(0)
# print(f"Shape of the first utterance: {first_utterance.shape}")
# print(f"Length of the corresponding transcript: {len(first_transcript)}")
# print(f"First 10 phonemes in the transcript: {first_transcript[:10]}")

# # Print the phoneme-to-index mapping for a few phonemes
# sample_phonemes = ['AH', 'ER', 'IY']
# for phoneme in sample_phonemes:
#     print(f"Index for phoneme '{phoneme}': {dataset.phonemeindex[phoneme]}")

# # Optionally, print some details about the data distribution, such as the average number of frames per utterance
# total_frames = sum(utterance.shape[0] for utterance, _ in map(dataset.get_utterance_and_transcript, range(len(dataset))))
# average_frames = total_frames / len(dataset)
# print(f"Average number of frames per utterance: {average_frames:.2f}")

# # If you're interested in the distribution of transcript lengths, you can also compute and print that
# total_transcript_length = sum(len(transcript) for _, transcript in map(dataset.get_utterance_and_transcript, range(len(dataset))))
# average_transcript_length = total_transcript_length / len(dataset)
# print(f"Average transcript length: {average_transcript_length:.2f}")


### Test Data

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):

  def __init__(self, root, PHONEMES):
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, 'mfcc')

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))

        self.PHONEMES = PHONEMES

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        self.mfccs = []

        # self.phonemeindex = {}

        # for i, phoneme in enumerate(self.PHONEMES):
        #     self.phonemeindex[phoneme] = i

        for i in range(len(self.mfcc_files)):
          # Load and normalize MFCCs
          mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
          mean = np.mean(mfcc)
          std = np.std(mfcc)
          mfcc = (mfcc - mean) / std  # Normalize MFCC
          self.mfccs.append(mfcc)


        self.length = len(self.mfccs)

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
  def __len__(self):
    return self.length

  def __getitem__(self, ind):
    mfcc = self.mfccs[ind]
    return torch.FloatTensor(mfcc)

  def collate_fn(self,batch):
    batch_mfcc = batch
    batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True) # TODO
    lengths_mfcc = [len(i) for i in batch_mfcc]
    return batch_mfcc_pad, torch.tensor(lengths_mfcc)

### Config - Hyperparameters

In [ ]:
BATCH_SIZE = 64 # Increase if your device can handle it

transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
config ={
    'epochs'            : 30,
    'batch_size'        : 64,   # Increase if your device can handle it
    'lr'                : 2e-3,
    'encoder dropout'   : 0.25,
    'decoder dropout'   : 0.15,
    'lstm dropout'      : 0.25,
    'factor'            : 0.5,
    'sc_mode'           : 'min',
    'patience'          : 3,
    'sc_threshold'      : 1e-3,
    'threshold mode'    : 'rel',
    'beam_width'        : 5 ,
    'test_beam_width'   : 2
}
root = '/content/11-785-s23-hw3p2'

### Data loaders

In [ ]:
# get me RAMMM!!!!
import gc
gc.collect()

905

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(root = '/content/11-785-s24-hw3p2/train-clean-100', PHONEMES=PHONEMES) #TODO
val_data = AudioDataset(root = '/content/11-785-s24-hw3p2/dev-clean', PHONEMES=PHONEMES) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root = '/content/11-785-s24-hw3p2/test-clean', PHONEMES=PHONEMES) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = DataLoader(
    train_data,
    batch_size=config['batch_size'],
    shuffle=True,  # Typically you shuffle the training data
    drop_last=True,  # Optionally drop the last incomplete batch
    collate_fn=train_data.collate_fn  # Assuming your AudioDataset has a collate_fn method
)

val_loader = DataLoader(
    val_data,
    batch_size=config['batch_size'],  # You might be able to use a larger batch size for validation if memory allows
    shuffle=False,  # No need to shuffle validation data
    drop_last=False,  # Don't drop the last batch in validation
    collate_fn=val_data.collate_fn  # Assuming the same collate_fn can be used for validation
)

test_loader = DataLoader(
    test_data,
    batch_size=config['batch_size'],
    shuffle=False,  # No need to shuffle test data
    drop_last=False,  # Don't drop the last batch in testing
    collate_fn=test_data.collate_fn  # Assuming your AudioDatasetTest has a similar collate_fn method
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 445
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1615, 27]) torch.Size([64, 202]) torch.Size([64]) torch.Size([64])


# NETWORK

## Basic

This is a basic block for understanding, you can skip this and move to pBLSTM one

In [ ]:
torch.cuda.empty_cache()

class Network(nn.Module):

    def __init__(self, input_size, out_size):

        super(Network, self).__init__()

        # Adding some sort of embedding layer or feature extractor might help performance.
        # self.embedding = ?

        # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = 256, num_layers = 1)

        self.classification = nn.Sequential(
            #TODO: Linear layer with in_features from the lstm module above and out_features = OUT_SIZE
            torch.nn.Linear(in_features = 256, out_features = out_size),
        )

        self.logSoftmax = nn.LogSoftmax(dim=1)

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints

        # Pack the padded sequence
        x_packed = nn.utils.rnn.pack_padded_sequence(x, lx, batch_first=True, enforce_sorted=False)

        # LSTM layer
        x_lstm, (h_n, c_n) = self.lstm(x_packed)

        # Unpack the sequence
        x_unpacked, lens_unpacked = pad_packed_sequence(x_lstm, batch_first=True)

        # Taking the last relevant output for classification, considering the varying lengths
        last_outputs = x_unpacked[torch.arange(x_unpacked.size(0)), lx-1]

        # Classification
        out = self.classification(last_outputs)

        # Log Softmax
        log_probs = self.logSoftmax(out)

        return log_probs

## Initialize Basic Network
(If trying out the basic Network)

In [ ]:
torch.cuda.empty_cache()
model = Network(input_size = 27, out_size = len(train_data.PHONEMES)).to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

                          Kernel Shape  Output Shape   Params Mult-Adds
Layer                                                                  
0_lstm                               -  [78016, 256]  291.84k  289.792k
1_classification.Linear_0    [256, 41]      [64, 41]  10.537k   10.496k
2_logSoftmax                         -      [64, 41]        -         -
-------------------------------------------------------------------------
                        Totals
Total params          302.377k
Trainable params      302.377k
Non-trainable params       0.0
Mult-Adds             300.288k


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_lstm,-,"[78016, 256]",291840.0,289792.0
1_classification.Linear_0,"[256, 41]","[64, 41]",10537.0,10496.0
2_logSoftmax,-,"[64, 41]",NaN,NaN


## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [ ]:
# Utils for network
torch.cuda.empty_cache()

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)

In [ ]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input?
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''

    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size=2*input_size,hidden_size=hidden_size,num_layers=1,batch_first=True,bidirectional=True)# TODO: Initialize a single layer bidirectional LSTM with the given input_size and hidden_size

    def forward(self, x_packed): # x_packed is a PackedSequence

        # TODO: Pad Packed Sequence

        # Call self.trunc_reshape() which downsamples the time steps of x and increases the feature dimensions as mentioned above
        # self.trunc_reshape will return 2 outputs. What are they? Think about what quantites are changing.
        # TODO: Pack Padded Sequence. What output(s) would you get?
        # TODO: Pass the sequence through bLSTM

        x_pad, x_seq_len = pad_packed_sequence(x_packed, batch_first= True)
        x_trunc, x_len = self.trunc_reshape(x_pad, x_seq_len)
        x_packed = pack_padded_sequence(x_trunc, x_len, batch_first= True, enforce_sorted= False)
        output, (h_n, c_n) = self.blstm(x_packed)

        # What do you return?

        return output

    def trunc_reshape(self, x, x_lens):
        # TODO: If you have odd number of timesteps, how can you handle it? (Hint: You can exclude them)
        # TODO: Reshape x. When reshaping x, you have to reduce number of timesteps by a downsampling factor while increasing number of features by the same factor
        # TODO: Reduce lengths by the same downsampling factor

        if x.shape[1]%2 != 0:
          x = x[:,:-1,:]
          x_lens -= 1
        x = x.reshape((x.shape[0],x.shape[1]//2, x.shape[2]*2))
        x_lens = x_lens//2
        return x, x_lens

### Encoder

In [ ]:
from torch.autograd import Variable
class LockedDropout(nn.Module):
  def __init__(self, dropout):
    super().__init__()
    self.dropout = dropout

  def forward(self, x):
    if not self.training or not self.dropout:
      return x
    x_unpacked, x_lens = pad_packed_sequence(x, batch_first= True)
    m = x.data.new(1, x_unpacked.size(1), x_unpacked.size(2)).bernoulli_(1 - self.dropout)
    mask = Variable(m, requires_grad= False) / (1 - self.dropout)
    mask = mask.expand_as(x_unpacked)
    out = mask * x_unpacked
    return pack_padded_sequence(out, lengths= x_lens, batch_first= True, enforce_sorted= False)

In [ ]:
class Encoder(torch.nn.Module):
    '''
    The Encoder takes utterances as inputs and returns latent feature representations
    '''
    def __init__(self, input_size, encoder_hidden_size):
        super(Encoder, self).__init__()


        self.embedding = torch.nn.Sequential(
            PermuteBlock(),
            torch.nn.Conv1d(in_channels= 27, out_channels= 27, kernel_size= 7, padding= 3),
            torch.nn.GELU(),
            PermuteBlock()
        )#TODO: You can use CNNs as Embedding layer to extract features. Keep in mind the Input dimensions and expected dimension of Pytorch CNN.
        self.BLSTMs = torch.nn.LSTM(input_size=27, hidden_size= encoder_hidden_size, num_layers=3, batch_first= True, bidirectional= True, dropout= config['lstm dropout'])

        self.pBLSTMs = torch.nn.Sequential( # How many pBLSTMs are required?
            # TODO: Fill this up with pBLSTMs - What should the input_size be?
            # Hint: You are downsampling timesteps by a factor of 2, upsampling features by a factor of 2 and the LSTM is bidirectional)
            # Optional: Dropout/Locked Dropout after each pBLSTM (Not needed for early submission)
            # https://github.com/salesforce/awd-lstm-lm/blob/dfd3cb0235d2caf2847a4d53e1cbd495b781b5d2/locked_dropout.py#L5
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size),
            LockedDropout(config['encoder dropout']),
            pBLSTM(encoder_hidden_size * 2, encoder_hidden_size),
            LockedDropout(config['encoder dropout'])
        )

    def forward(self, x, x_lens):
        # Where are x and x_lens coming from? The dataloader
        #TODO: Call the embedding layer
        # TODO: Pack Padded Sequence
        # TODO: Pass Sequence through the pyramidal Bi-LSTM layer
        # TODO: Pad Packed Sequence
        x_out = self.embedding(x)
        x_out = pack_padded_sequence(x_out, x_lens, batch_first= True, enforce_sorted= False)
        x_blstm, (h_n, c_n) = self.BLSTMs(x_out)
        x_pblstm = self.pBLSTMs(x_blstm)
        encoder_outputs, encoder_lens = pad_packed_sequence(x_pblstm, batch_first= True)
        # Remember the number of output(s) each function returns

        return encoder_outputs, encoder_lens

### Decoder

In [ ]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(2*embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            torch.nn.Linear(2 * embed_size, 2 * embed_size),
            torch.nn.GELU(),
            torch.nn.Dropout(config['decoder dropout']),
            torch.nn.Linear(2 * embed_size, 2 * embed_size),
            torch.nn.GELU(),
            torch.nn.Dropout(config['decoder dropout']),
            torch.nn.Linear(2 * embed_size, output_size)
            )

        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification
        mlp_out = self.mlp(encoder_out)
        out = self.softmax(mlp_out)
        return out

In [ ]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        self.augmentations  = torch.nn.Sequential(
            #TODO Add Time Masking/ Frequency Masking
            #Hint: See how to use PermuteBlock() function defined above
        )
        self.encoder        = Encoder(input_size, embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size, output_size)# TODO: Initialize Decoder



    def forward(self, x, lengths_x):

        if self.training:
            x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [ ]:
import torchsummaryX

In [ ]:
model = ASRModel(
    input_size  = 27,
    embed_size  = 256,#TODO
    output_size = len(PHONEMES)
).to(device)
print(model)
summary(model, x.to(device), lx)

ASRModel(
  (augmentations): Sequential()
  (encoder): Encoder(
    (embedding): Sequential(
      (0): PermuteBlock()
      (1): Conv1d(27, 27, kernel_size=(7,), stride=(1,), padding=(3,))
      (2): GELU(approximate='none')
      (3): PermuteBlock()
    )
    (BLSTMs): LSTM(27, 256, num_layers=3, batch_first=True, dropout=0.25, bidirectional=True)
    (pBLSTMs): Sequential(
      (0): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (1): LockedDropout()
      (2): pBLSTM(
        (blstm): LSTM(1024, 256, batch_first=True, bidirectional=True)
      )
      (3): LockedDropout()
    )
  )
  (decoder): Decoder(
    (mlp): Sequential(
      (0): PermuteBlock()
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): PermuteBlock()
      (3): Linear(in_features=512, out_features=512, bias=True)
      (4): GELU(approximate='none')
      (5): Dropout(p=0.15, inplace=False)
      (6): Linear(in_features=

,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_augmentations,-,"[64, 1615, 27]",NaN,NaN
1_encoder.embedding.PermuteBlock_0,-,"[64, 27, 1615]",NaN,NaN
2_encoder.embedding.Conv1d_1,"[27, 27, 7]","[64, 27, 1615]",5130.0,8241345.0
3_encoder.embedding.GELU_2,-,"[64, 27, 1615]",NaN,NaN
4_encoder.embedding.PermuteBlock_3,-,"[64, 1615, 27]",NaN,NaN
5_encoder.LSTM_BLSTMs,-,"[78016, 512]",3737600.0,3725312.0
6_encoder.pBLSTMs.0.LSTM_blstm,-,"[38959, 512]",2625536.0,2621440.0
7_encoder.pBLSTMs.LockedDropout_1,-,"[38959, 512]",NaN,NaN
8_encoder.pBLSTMs.2.LSTM_blstm,-,"[19434, 512]",2625536.0,2621440.0


# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [ ]:
#TODO


criterion = nn.CTCLoss(blank= CMUdict.index(''))# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

# optimizer =  torch.optim.AdamW(...) # What goes in here?
optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(labels=LABELS, beam_width=config['beam_width'], num_processes= 4, log_probs_input=True)#TODO

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode= config['sc_mode'], factor= config['factor'], patience= config['patience'], threshold= config['sc_threshold'], threshold_mode= config['threshold mode'],  verbose=True)#TODO

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):

    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    #output = torch.permute(output, (1,0,2))
    output, _, _, out_seq_len = decoder.decode(output, seq_lens= output_lens)

    pred_strings                    = []

    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        logits=output[i][0][:out_seq_len[i][0]]
        # print('index',i)
        # print('logits',max(logits))
        try: pred_strings.append(''.join([PHONEME_MAP[i] for i in logits]))
        except:
          print(logits)
          return
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers

    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)

    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch

        pred_string = pred_strings[i]#TODO
        tmp = ""
        for l in label[i][:label_lens[i]]:
          #print(PHONEME_MAP[l.int()])
          tmp += PHONEME_MAP[l.int()]

        label_string = ''.join(tmp)#TODO
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return dist

In [ ]:
# model = ASRModel(input_size = 27, embed_size = 256, output_size = len(PHONEMES))  # Use appropriate parameters
# decoder = CTCBeamDecoder(LABELS)  # Instantiate your CTC decoder

# # Assuming 'input_data' is your input and 'input_lengths' are the lengths of your input sequences
# # You might need to preprocess your input data to match your model's requirements

# # Run the input data through your model
# decoder_out, encoder_lens = model(x, torch.tensor([x.shape[1]]))

# # Now 'decoder_out' and 'encoder_lens' are defined and you can call 'decode_prediction'
# decode_prediction(decoder_out, encoder_lens, decoder, PHONEME_MAP=LABELS)

# Test Implementation

In [ ]:
# test code to check shapes

model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)

    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))

    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(1.0794, device='cuda:0', grad_fn=<MeanBackward0>)


ValueError: cannot create std::vector larger than max_size()

# WandB

You will need to fetch your api key from wandb.ai

In [ ]:
import wandb
wandb.login(key="4c86d0f2bd3d0f4cd0c81ea3862c4119ec70e8e4")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
run = wandb.init(
    name = "VS-final2", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: vedantjh13 (idl1). Use `wandb login --relogin` to force relogin


# Train Functions

In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [ ]:
def save_model(model, optimizer, scheduler, metric, epoch, path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
         path
    )

def load_model(path, model, metric= 'valid_acc', optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    epoch   = checkpoint['epoch']
    metric  = checkpoint[metric]

    return [model, optimizer, scheduler, epoch, metric]

In [ ]:
last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf")  # if you're restarting from some checkpoint, use what you saw there.

# Corrected paths
epoch_model_path = "/content/Epoch_Model_Path"  # Path for saving models after each epoch
best_model_path = "/content/Best_Model_Path"  # Path for saving the best model

In [ ]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

# model, optimizer, scheduler, last_epoch, valid_dist = load_model(best_model_path, model, 'valid_dist', optimizer, scheduler)

# best_lev_dist = valid_dist

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss              = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
    scheduler.step(valid_dist)

    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })

    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, epoch_model_path)
    wandb.save(epoch_model_path)
    print("Saved epoch model")

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, best_model_path)
        wandb.save(best_model_path)
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/30


Val:   0%|          | 0/43 [00:00<?, ?it/s]

ValueError: cannot create std::vector larger than max_size()

# Generate Predictions and Submit to Kaggle

In [ ]:
TEST_BEAM_WIDTH = 5 #TODO

test_decoder    = CTCBeamDecoder(labels=LABELS, beam_width=TEST_BEAM_WIDTH, log_probs_input=True)#TODO
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)
        # h = torch.permute(h, (1, 0, 2))
    prediction_string= decode_prediction(h, lh, test_decoder, LABELS)# TODO call decode_prediction
    #TODO save the output in results array.
    results.extend(prediction_string)

    del x, lx, h, lh
    torch.cuda.empty_cache()

In [ ]:
data_dir = f"{root}/test-clean/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [ ]:
!kaggle competitions submit -c hw3p2asr-s24 -f submission.csv -m "I made it!"